In [2]:
import pandas as pd
import numpy as np
import json

marker = "CZ_2"
csv_path = "/Users/bytedance/Documents/code/data/【网络视频 - CZ】交付-验收数据 - 第二批 50条 3.12（打回重标）.csv"

df = pd.read_csv(csv_path)
# modify the column name
for col in df.columns:
    df = df.rename(columns={col: col.strip().strip("\\n")})

# replace nan with None
df = df.replace(np.nan, None)

video = None
video_list = []

for row in df.iterrows():
    row = row[1]
    if row["视频序号"]:
        if video:
            video_list.append(video)
        video = {}
        video["video_id"] = marker + "_" + str(int(row["视频序号"]))
        video["video_url"] = row["视频_url"]
        video["video_duration"] = row["视频长度（min）"]
        video["video_type"] = row["视频类型"]
        video["qa_list"] = []
    qa = {
        "question": row["问题"],
        "answer": row["答案"],
        "question_type": row["问题类型"],
        "knowledge": row["若涉及通用知识推理，则写出推理步骤（知识点）"],
        "reasoning": row["依据（解题思路）"],
    }
    video["qa_list"].append(qa)

video_list.append(video)

print(len(video_list))
print(video_list[0])

# save the video_list to a json file
with open(f"data/annotations/{marker}.json", "w") as f:
    json.dump(video_list, f, ensure_ascii=False, indent=4)

50
{'video_id': 'CZ_2_1', 'video_url': 'https://www.youtube.com/watch?v=PnvZZwlN2yk', 'video_duration': '29:04', 'video_type': '综艺 - 游戏', 'qa_list': [{'question': 'Is Stewart Thompson a person who pursues a high-quality life?', 'answer': 'Yes.', 'question_type': '多线索推理,人物属性建模', 'knowledge': None, 'reasoning': '从Stewart讲述自己旅行方式倾向于商务舱可以判断。'}, {'question': 'What decade of age is J.F. Harris approximately in?', 'answer': 'He is approximately in his forties.', 'question_type': '多线索推理,人物属性建模', 'knowledge': None, 'reasoning': 'J.F. Harris提到过自己2003年曾经在芝加哥，后来又说20岁左右在芝加哥，综合推理得知现在J.F. Harris现在大概四十多岁。'}, {'question': 'Who had a different opinion from the others when voting out the person who was not a millionaire in the first round?', 'answer': 'Aaron.', 'question_type': '多跳推理,多模态/多语言推理', 'knowledge': None, 'reasoning': '首先需要定位到第一轮投票这个片段，再结合视觉听觉对五个人的发言判断。'}, {'question': 'What economic conditions do the families of children who have a trust fund usually come from?', 'answer': 'The family possesses

In [ ]:
annotaions_path = "data/annotations/CZ_2.json"
marker = annotaions_path.split("/")[-1].split(".")[0]
with open(annotaions_path, "r") as f:
    data = json.load(f)

count = 0

for video in data:
    url = video["video_url"]
    if not url.startswith("https://www.youtube.com/"):
        count += 1
        with open(f"data/annotations/{marker}_error_video_url.txt", "a") as f:
            f.write(url + "\n")

print(f"Total error video urls: {count}")